In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import h5py
from scipy.integrate import solve_ivp
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm
import random

def wave_equation(t, U, c, dx):
    U = U.reshape((-1, 2))
    dUdt = np.zeros_like(U)
    dUdt[:, 0] = U[:, 1]
    dUdt[1:-1, 1] = c**2 * (U[:-2, 0] - 2 * U[1:-1, 0] + U[2:, 0]) / dx**2
    return dUdt.flatten()

def newData(c, f):
    x_min, x_max = -1.0, 1.0  # Spatial domain
    t_min, t_max = 0.0, 2.0   # Temporal domain
    num_x, num_t = 1024, 201  # Grid resolution
    dx = (x_max - x_min) / (num_x - 1)
    dt = 0.9 * dx / c  # CFL condition for stability

    x = np.linspace(x_min, x_max, num_x)
    t_eval = np.linspace(t_min, t_max, num_t)
    u0 = np.exp(-50 * (x**2))
    v0 = np.zeros_like(u0)
    U0 = np.vstack((u0, v0)).T.flatten()
    
    sol = solve_ivp(wave_equation, [t_min, t_max], U0, t_eval=t_eval, args=(c, dx), method='RK45')
    U_clean = sol.y[:num_x, :]
    U_noise = U_clean + skewnorm.rvs(a=1, scale=0.2, size=U_clean.shape)
    
    grp = f.create_group(f"wave_solution_{c}")
    grp.create_dataset("clean", data=U_clean)
    grp.create_dataset("noisy", data=U_noise)
    grp.create_dataset("c", data=c)
    
    print(f"Data for c = {c} generated and added to HDF5 file")

with h5py.File("wave_solutions_new.h5", 'w') as f:
    for c in range(1, 963):
        newData(c, f)

def visualize_wave(xcrd, data, path):
    fig, ax = plt.subplots()
    ims = []
    for i in tqdm(range(data.shape[1])):
        im, = ax.plot(xcrd, data[:, i], animated=True, color="blue")
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)
    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

with h5py.File("/kaggle/working/wave_solutions_new.h5", "r") as f:
    x = np.linspace(-1.0, 1.0, 1024)
    l = random.choices(list(f.keys()), k=3)
    for i in l:
        U_clean = f[i]['clean'][:]
        U_noisy = f[i]['noisy'][:]
        # visualize_wave(x, U_clean, f"gifs/{i}.gif")
        # visualize_wave(x, U_noisy, f"gifs/{i}_noisy.gif")


Data for c = 1 generated and added to HDF5 file
Data for c = 2 generated and added to HDF5 file
Data for c = 3 generated and added to HDF5 file
Data for c = 4 generated and added to HDF5 file
Data for c = 5 generated and added to HDF5 file
Data for c = 6 generated and added to HDF5 file
Data for c = 7 generated and added to HDF5 file
Data for c = 8 generated and added to HDF5 file
Data for c = 9 generated and added to HDF5 file
Data for c = 10 generated and added to HDF5 file
Data for c = 11 generated and added to HDF5 file
Data for c = 12 generated and added to HDF5 file
Data for c = 13 generated and added to HDF5 file
Data for c = 14 generated and added to HDF5 file
Data for c = 15 generated and added to HDF5 file
Data for c = 16 generated and added to HDF5 file
Data for c = 17 generated and added to HDF5 file
